In [2]:
from bokeh.plotting import figure
from bokeh.layouts import row, column, widgetbox, gridplot

from bokeh.io import curdoc
from bokeh.io import output_notebook, show, output_file

from bokeh.models import ColumnDataSource, HoverTool, TapTool, Range1d, OpenURL
from bokeh.models import LinearColorMapper , ColorBar
from bokeh.models.widgets import Select, Slider


import numpy as np

In [3]:
# =============================================
# THIS comes from INTERFACE
#
selected_exposure = '00000003'# args['exposure']
selected_arm = 'z' # args['arm']
selected_spectrograph =  7 # args['spectrograph']

# =============================================
# THIS comes from QLF.CFG
#
night = '20190101'

# ============================================
#  THIS READ yaml files
#
from scalar_metrics_v02 import LoadMetrics

cam = selected_arm+str(selected_spectrograph)
exp = selected_exposure # intentionaly redundant
lm = LoadMetrics(cam, exp, night);
metrics, tests  = lm.metrics, lm.tests 

# =============================================
# THIS is only to simplify the code understanding
#
countpix  = metrics['countpix']
getbias   = metrics['getbias']
getrms    = metrics['getrms']
xwsigma   = metrics['xwsigma']
countbins = metrics['countbins']
integ     = metrics['integ']
skycont   = metrics['skycont']
skypeak   = metrics['skypeak']
skyresid  = metrics['skyresid']
snr       = metrics['snr']

countpix loaded
/home/foliveira/quicklook/spectro/redux/exposures/20190101/00000003/ql-countpix-z7-00000003.yaml
getbias loaded
/home/foliveira/quicklook/spectro/redux/exposures/20190101/00000003/ql-getbias-z7-00000003.yaml
getrms loaded
/home/foliveira/quicklook/spectro/redux/exposures/20190101/00000003/ql-getrms-z7-00000003.yaml
xwsigma loaded
/home/foliveira/quicklook/spectro/redux/exposures/20190101/00000003/ql-xwsigma-z7-00000003.yaml
countbins loaded
/home/foliveira/quicklook/spectro/redux/exposures/20190101/00000003/ql-countbins-z7-00000003.yaml
integ loaded
/home/foliveira/quicklook/spectro/redux/exposures/20190101/00000003/ql-integ-z7-00000003.yaml
skycont loaded
/home/foliveira/quicklook/spectro/redux/exposures/20190101/00000003/ql-skycont-z7-00000003.yaml
skypeak loaded
/home/foliveira/quicklook/spectro/redux/exposures/20190101/00000003/ql-skypeak-z7-00000003.yaml
skyresid loaded
/home/foliveira/quicklook/spectro/redux/exposures/20190101/00000003/ql-skyresid-z7-00000003.yaml

In [4]:
#Additional imports:
from bokeh.layouts import widgetbox
from bokeh.models.widgets import Div



hist_tooltip = """
    <div>
        <div>
            <span style="font-size: 12px; font-weight: bold; color: #303030;">Frequency: </span>
            <span style="font-size: 13px; color: #515151">@hist</span>
        </div>
        <div>
            <span style="font-size: 12px; font-weight: bold; color: #303030;">sigma: </span>
            <span style="font-size: 13px; color: #515151;">[@left, @right]</span>
        </div>
    </div>
"""
name_hi = 'NBINSHIGH'
name_med = 'NBINSMED'
name_low = 'NBINSLOW'

def bins_doane(data):
    # Dane's rule of thumb for bins
    ndata = len(data)
    mean = np.mean(data)
    sigma = np.std(data)
    b = sum([ (x - mean)**3 for x in data])
    b = b/sum([ (x - mean)**2 for x in data])**(1.5)
    try:
        return int(round(np.log2(ndata) + 1 + np.log2((1.+b)/(sigma*b))))
    except:
        return int(round(np.log2(ndata) + 1 ))
    
bins_hi = bins_doane(countbins[name_hi])
bins_med = bins_doane(countbins[name_med])
bins_low = bins_doane(countbins[name_low])# formely: 17
hover = HoverTool(tooltips=hist_tooltip)
hover2 = HoverTool(tooltips=hist_tooltip)
hover3 = HoverTool(tooltips=hist_tooltip)


# ===
hist_hi, edges_hi = np.histogram(countbins[name_hi], bins = bins_hi)
source_hi = ColumnDataSource(data={
    'hist':hist_hi,
    'bottom':[0] *len(hist_hi),
    'left':edges_hi[:-1],
    'right':edges_hi[1:]
})

# ===
hist_med, edges_med = np.histogram(countbins[name_med], bins = bins_med)
source_med = ColumnDataSource(data={
    'hist':hist_med,
    'bottom':[0] *len(hist_med),
    'left':edges_med[:-1],
    'right':edges_med[1:]
})

# ===
hist_low, edges_low = np.histogram(countbins[name_low], bins = bins_low)
source_low = ColumnDataSource(data={
    'hist':hist_low,
    'bottom':[0] *len(hist_low),
    'left':edges_low[:-1],
    'right':edges_low[1:]
})



phi = figure(title='NBINSHI',tools=[hover,"pan,wheel_zoom,box_zoom,reset"],
           y_axis_label='Frequency', x_axis_label='COUNTBINS', background_fill_color="white")

phi.quad(top='hist', bottom='bottom', left='left', right='right',
       source=source_hi, 
        fill_color="dodgerblue", line_color="black", alpha=0.8,
       hover_fill_color='blue', hover_line_color='black', hover_alpha=0.8)

pmed = figure(title='NBINSMED',tools=[hover2,"pan,wheel_zoom,box_zoom,reset"],
           y_axis_label='Frequency', x_axis_label='COUNTBINS', background_fill_color="white")

pmed.quad(top='hist', bottom='bottom', left='left', right='right',
       source=source_med, 
        fill_color="lightgreen", line_color="black", alpha=0.8,
       hover_fill_color='green', hover_line_color='black', hover_alpha=0.8)


plow = figure(title='NBINSLOW',tools=[hover3,"pan,wheel_zoom,box_zoom,reset"],
           y_axis_label='Frequency', x_axis_label='COUNTBINS', background_fill_color="white")


plow.quad(top='hist', bottom='bottom', left='left', right='right',
       source=source_low, fill_color="tomato", line_color="black", alpha=0.8,
       hover_fill_color='red', hover_line_color='black', hover_alpha=0.8)

#plow.legend.location = "top_left"

html_str="""
<style>
    table {
        font-family: arial, sans-serif;
        font-size: 20px;
        border-collapse: collapse;
        width: 100%;
    }

    td, th {
        border: 1px solid #dddddd;
        text-align: center;
        padding: 8px;
    }
    tr:nth-child(even) {
        background-color: #dddddd;
                text-align:center;
    }
    tr:{text-align:center;}
</style>

<div  style="text-align:center;">
<table>
  <tr>
    <th>Parameter</th>
    <th>Value</th>
  </tr>
  <tr>
    <td>CUTLO</td>
    <td>param0</td>
  </tr>
  <tr>
    <td>CUTMED</td>
    <td>param1</td>
  </tr>
  <tr>
    <td>CUTHI</td>
    <td>param2</td>
  </tr>

</table>
</div>

"""
txt_keys=['CUTLO','CUTMED','CUTHI']
for i in range(3):
    html_str=html_str.replace('%s%s'%("param",str(i)), str(tests['countbins'][txt_keys[i]]) )

div=Div(text=html_str, 
        width=400, height=200)


my_layout = gridplot( [phi,pmed,plow,div], ncols=2, plot_width=600, plot_height=600)
 

output_notebook()
show(my_layout)
#output_notebook('hist_countbins.html')


/home/foliveira/miniconda3/envs/quicklook/lib/python3.5/site-packages/ipykernel_launcher.py:31: RuntimeWarning: invalid value encountered in log2


Loading BokehJS ...

## DropDown menu'

In [5]:
#Additional imports:
from bokeh.layouts import widgetbox
from bokeh.models.widgets import Div, tab



hist_tooltip = """
    <div>
        <div>
            <span style="font-size: 12px; font-weight: bold; color: #303030;">Frequency: </span>
            <span style="font-size: 13px; color: #515151">@hist</span>
        </div>
        <div>
            <span style="font-size: 12px; font-weight: bold; color: #303030;">sigma: </span>
            <span style="font-size: 13px; color: #515151;">[@left, @right]</span>
        </div>
    </div>
"""
name_hi = 'NBINSHIGH'
name_med = 'NBINSMED'
name_low = 'NBINSLOW'

def bins_doane(data):
    # Dane's rule of thumb for bins
    ndata = len(data)
    mean = np.mean(data)
    sigma = np.std(data)
    b = sum([ (x - mean)**3 for x in data])
    b = b/sum([ (x - mean)**2 for x in data])**(1.5)
    try:
        return int(round(np.log2(ndata) + 1 + np.log2((1.+b)/(sigma*b))))
    except:
        return int(round(np.log2(ndata) + 1 ))
    
bins_hi = bins_doane(countbins[name_hi])
bins_med = bins_doane(countbins[name_med])
bins_low = bins_doane(countbins[name_low])# formely: 17
hover = HoverTool(tooltips=hist_tooltip)
hover2 = HoverTool(tooltips=hist_tooltip)
hover3 = HoverTool(tooltips=hist_tooltip)


# ===
hist_hi, edges_hi = np.histogram(countbins[name_hi], bins = bins_hi)
source_hi = ColumnDataSource(data={
    'hist':hist_hi,
    'bottom':[0] *len(hist_hi),
    'left':edges_hi[:-1],
    'right':edges_hi[1:]
})

# ===
hist_med, edges_med = np.histogram(countbins[name_med], bins = bins_med)
source_med = ColumnDataSource(data={
    'hist':hist_med,
    'bottom':[0] *len(hist_med),
    'left':edges_med[:-1],
    'right':edges_med[1:]
})

# ===
hist_low, edges_low = np.histogram(countbins[name_low], bins = bins_low)
source_low = ColumnDataSource(data={
    'hist':hist_low,
    'bottom':[0] *len(hist_low),
    'left':edges_low[:-1],
    'right':edges_low[1:]
})



phi = figure(title='NBINSHI',tools=[hover,"pan,wheel_zoom,box_zoom,reset"],
           y_axis_label='Frequency', x_axis_label='COUNTBINS', background_fill_color="white")

phi.quad(top='hist', bottom='bottom', left='left', right='right',
       source=source_hi, 
        fill_color="dodgerblue", line_color="black", alpha=0.8,
       hover_fill_color='blue', hover_line_color='black', hover_alpha=0.8)

pmed = figure(title='NBINSMED',tools=[hover2,"pan,wheel_zoom,box_zoom,reset"],
           y_axis_label='Frequency', x_axis_label='COUNTBINS', background_fill_color="white")

pmed.quad(top='hist', bottom='bottom', left='left', right='right',
       source=source_med, 
        fill_color="lightgreen", line_color="black", alpha=0.8,
       hover_fill_color='green', hover_line_color='black', hover_alpha=0.8)


plow = figure(title='NBINSLOW',tools=[hover3,"pan,wheel_zoom,box_zoom,reset"],
           y_axis_label='Frequency', x_axis_label='COUNTBINS', background_fill_color="white")


plow.quad(top='hist', bottom='bottom', left='left', right='right',
       source=source_low, fill_color="tomato", line_color="black", alpha=0.8,
       hover_fill_color='red', hover_line_color='black', hover_alpha=0.8)

#plow.legend.location = "top_left"

html_str="""
<style>
    table {
        font-family: arial, sans-serif;
        font-size: 20px;
        border-collapse: collapse;
        width: 100%;
    }

    td, th {
        border: 1px solid #dddddd;
        text-align: center;
        padding: 8px;
    }
    tr:nth-child(even) {
        background-color: #dddddd;
                text-align:center;
    }
    tr:{text-align:center;}
</style>

<div  style="text-align:center;">
<table>
  <tr>
    <th>Parameter</th>
    <th>Value</th>
  </tr>
  <tr>
    <td>CUTLO</td>
    <td>param0</td>
  </tr>
  <tr>
    <td>CUTMED</td>
    <td>param1</td>
  </tr>
  <tr>
    <td>CUTHI</td>
    <td>param2</td>
  </tr>

</table>
</div>

"""
txt_keys=['CUTLO','CUTMED','CUTHI']
for i in range(3):
    html_str=html_str.replace('%s%s'%("param",str(i)), str(tests['countbins'][txt_keys[i]]) )

div=Div(text=html_str, 
        width=400, height=200)


my_layout = gridplot( [phi,pmed,plow,div], ncols=2, plot_width=600, plot_height=600)
 
####
# Define a callback function: update_plot
def update_plot(attr, old, new):
    # If the new Selection is 'female_literacy', update 'y' to female_literacy
    if new == 'female_literacy': 
        source.data = {
            'x' : fertility,
            'y' : female_literacy
        }
    # Else, update 'y' to population
    else:
        source.data = {
            'x' : fertility,
            'y' : population
        }

# Create a dropdown Select widget: select    
select = Select(title="distribution", options=['female_literacy', 'population'], value='female_literacy')

# Attach the update_plot callback to the 'value' property of select
select.on_change('value', update_plot)
###

output_notebook()
show(my_layout)
#output_notebook('hist_countbins.html')


/home/foliveira/miniconda3/envs/quicklook/lib/python3.5/site-packages/ipykernel_launcher.py:31: RuntimeWarning: invalid value encountered in log2


Loading BokehJS ...

## Infos!

In [225]:
p = figure(title="Periodic Table (omitting LA and AC Series)", plot_width=1000, plot_height=450,
           tools="", toolbar_location=None,
          )#x_range=groups, y_range=list(reversed(periods)))
#style="font-size: 14px; font-weight: bold; color: blue;float:left;",
html_str2="""
<body style="color:"red";">
<table>
  <tr>
    <th>Parameter</th>
    <th>Value</th>
  </tr>
  <tr>
    <td>CUTLO</td>
    <td>param0</td>
  </tr>
  <tr>
    <td>CUTMED</td>
    <td>param1</td>
  </tr>
  <tr>
    <td>CUTHI</td>
    <td>param2</td>
  </tr>

</table>
</body>
"""
html_str="""
<style>
div.blueTable {
  background-color: #EEEEEE;
  width: 100%;
  text-align: center;
}
.divTable.blueTable .divTableCell, .divTable.blueTable .divTableHead {
  padding: 3px 2px;
}
.divTable.blueTable .divTableBody .divTableCell {
  font-size: 13px;
}
.divTable.blueTable .divTableRow:nth-child(even) {
  background: #DCDCDE;
}
.blueTable .tableFootStyle {
  font-size: 14px;
}
.blueTable .tableFootStyle .links {
	 text-align: right;
}
.blueTable .tableFootStyle .links a{
  display: inline-block;
  background: #1C6EA4;
  color: #FFFFFF;
  padding: 2px 8px;
  border-radius: 5px;
}
.blueTable.outerTableFooter {
  border-top: none;
}
.blueTable.outerTableFooter .tableFootStyle {
  padding: 3px 5px; 
}
/* DivTable.com */
.divTable{ display: table; }
.divTableRow { display: table-row; }
.divTableHeading { display: table-header-group;}
.divTableCell, .divTableHead { display: table-cell;}
.divTableHeading { display: table-header-group;}
.divTableFoot { display: table-footer-group;}
.divTableBody { display: table-row-group;}
</style>

<div class="divTable blueTable">
    <div class="divTableHeading">
        <div class="divTableRow">
        </div>
    </div>
<div class="divTableBody">
    <div class="divTableRow">
        <div class="divTableCell">cell1_1</div>
        <div class="divTableCell">cell2_1</div>
    </div>
    <div class="divTableRow">
        <div class="divTableCell">cell1_2</div>
        <div class="divTableCell">cell2_2</div>
    </div>
    <div class="divTableRow">
        <div class="divTableCell">cell1_3</div>
        <div class="divTableCell">cell2_3</div>
    </div>
</div>
</div>
"""

div = Div(text=html_str, width=200, height=200); show(widgetbox(div))

In [233]:
pref="""
<style>
div.blueTable {
  background-color: #EEEEEE;
  width: 100%;
  text-align: center;
}
.divTable.blueTable .divTableCell, .divTable.blueTable .divTableHead {
  padding: 3px 2px;
}
.divTable.blueTable .divTableBody .divTableCell {
  font-size: 13px;
}
.divTable.blueTable .divTableRow:nth-child(even) {
  background: #DCDCDE;
}
.blueTable .tableFootStyle {
  font-size: 14px;
}
.blueTable .tableFootStyle .links {
	 text-align: right;
}
.blueTable .tableFootStyle .links a{
  display: inline-block;
  background: #1C6EA4;
  color: #FFFFFF;
  padding: 2px 8px;
  border-radius: 5px;
}
.blueTable.outerTableFooter {
  border-top: none;
}
.blueTable.outerTableFooter .tableFootStyle {
  padding: 3px 5px; 
}
/* DivTable.com */
.divTable{ display: table; }
.divTableRow { display: table-row; }
.divTableHeading { display: table-header-group;}
.divTableCell, .divTableHead { display: table-cell;}
.divTableHeading { display: table-header-group;}
.divTableFoot { display: table-footer-group;}
.divTableBody { display: table-row-group;}
</style>

<div class="divTable blueTable">
    <div class="divTableHeading">
        <div class="divTableRow">
        </div>
    </div>
<div class="divTableBody">"""


cell ="""<div class="divTableRow">
        <div class="divTableCell">col1</div>
        <div class="divTableCell">col2</div>
    </div>
"""   

suf = """</div></div>"""
html_str = pref+cell+cell+cell+suf
div = Div(text=html_str, width=200, height=200); show(widgetbox(div))

In [274]:
c1 = ['a','b','c', 'd']
c2 = [1,2,3,4]

def html_table(list_of_list, align="center"):
    """ insert list of list """
    ll = list_of_list
    columns = len(ll)
    style="""<style>
    div.blueTable {
      background-color: #EEEEEE;
      width: 100%;
      text-align: center;
    }
    .divTable.blueTable .divTableCell, .divTable.blueTable .divTableHead {
      padding: 3px 2px;
    }
    .divTable.blueTable .divTableBody .divTableCell {
      font-size: 13px;
    }
    .divTable.blueTable .divTableRow:nth-child(even) {
      background: #DCDCDE;
    }
    .blueTable .tableFootStyle {
      font-size: 14px;
    }
    .blueTable .tableFootStyle .links {
         text-align: right;
    }
    .blueTable .tableFootStyle .links a{
      display: inline-block;
      background: #1C6EA4;
      color: #FFFFFF;
      padding: 2px 8px;
      border-radius: 5px;
    }
    .blueTable.outerTableFooter {
      border-top: none;
    }
    .blueTable.outerTableFooter .tableFootStyle {
      padding: 3px 5px; 
    }

    .divTable{ display: table; }
    .divTableRow { display: table-row; }
    .divTableHeading { display: table-header-group;}
    .divTableCell, .divTableHead { display: table-cell;}
    .divTableHeading { display: table-header-group;}
    .divTableFoot { display: table-footer-group;}
    .divTableBody { display: table-row-group;}
    </style>""" 


    aux0 = """<div class="divTableRow">\n"""
    aux1 = """  <div class="divTableCell">col</div>\n"""
    aux2 ="""</div>"""
    aux = """"""
    for i in  range(len(ll)):
        aux = aux+aux1.replace("col", "col"+str(i))
    aux=aux0+ aux+aux2
    print (aux)
    cell=""""""
    for i in range(len(ll[0])):
        cell = cell+aux
        for j in range(len(ll)):
            cell = cell.replace("col"+str(j),str(ll[j][i]))

    html_str = style+ pref+cell+suf
    return html_str

div = Div(text=html_table(ll), width=200, height=200); show(widgetbox(div))

<div class="divTableRow">
  <div class="divTableCell">col0</div>
  <div class="divTableCell">col1</div>
  <div class="divTableCell">col2</div>
</div>


In [268]:
c3 = [333,333,332,3323]
ll=[c1,c2,c3]
print(ll)

[['a', 'b', 'c', 'd'], [1, 2, 3, 4], [333, 333, 332, 3323]]


In [258]:
type(html_table(ll))

<div class="divTableRow">
  <div class="divTableCell">col0</div>
  <div class="divTableCell">col1</div>
</div>


str

In [ ]:
 countbins.keys()#['NPIXHIGH']# (blue), countbins[‘NPIXMED’] (green) and countbins[‘NPIXLOW’] (orang

In [ ]:
countbins['NBINSHIGH'][:3]

In [ ]:
sname='NBINSHIGH'
mean=np.mean(countbins['NBINSMED'])
b = sum([ (x - mean)**3 for x in countbins[sname]])
b = b/sum([ (x - mean)**2 for x in countbins[sname]])**(1.5)
print( b)

## Comentários:

- separar os plots? 
- Quando ativar o modo Log
- Critérios de Binagem

In [28]:
for i in ['preproc', 'extract', 'fiberfl', 'skysubs']:
    print('----'*13,'\n', i)
    lm.PARTIALstep_color(i)
    print('\n')

---------------------------------------------------- 
 preproc
{'getbias': 'False', 'skyresid': 'False', 'skypeak': 'False', 'countbins': 'False', 'xwsigma': 'False', 'skycont': 'False', 'getrms': 'False', 'snr': 'False', 'countpix': 'False', 'integ': 'False'}
countpix False
getbias False
getrms False
Steps_status: ['ALARM', 'NORMAL', 'ALARM']
red


---------------------------------------------------- 
 extract
{'getbias': 'False', 'skyresid': 'False', 'skypeak': 'False', 'countbins': 'False', 'xwsigma': 'False', 'skycont': 'False', 'getrms': 'False', 'snr': 'False', 'countpix': 'False', 'integ': 'False'}
countbins False
Steps_status: ['ALARM']
red


---------------------------------------------------- 
 fiberfl
{'getbias': 'False', 'skyresid': 'False', 'skypeak': 'False', 'countbins': 'False', 'xwsigma': 'False', 'skycont': 'False', 'getrms': 'False', 'snr': 'False', 'countpix': 'False', 'integ': 'False'}
integ False
skycont False
skypeak False
skyresid False
Steps_status: ['NORMAL', 